In [250]:
import numpy as np
from ctypes import c_uint8, c_uint16, c_int8

In [7]:
instrucoes = {
    0x0: 2, 0x1: 2, 0x2: 2, 0x3: 1,
    0x4: 2, 0x5: 2, 0x6: 2, 0x7: 2,
    0x8: 2, 0x9: 2, 0xA: 2, 0xB: 1,
    0xC: 1,
}

In [8]:
instrucoes[0x0]

2

In [51]:
a = c_uint8(512)

In [52]:
256 * 2

512

In [53]:
a.value = 10000

In [55]:
a.value & 0xFFF

16

In [63]:
a.value >> 8

0

In [191]:
a = np.zeros(4096, dtype=np.uint8)

In [118]:
a[1] = 100

In [128]:
a[0] << 8 | a[1]

356

In [129]:
a = np.uint8(100)

In [134]:
a = np.uint8(257)

In [138]:
a & 0xFFf

1

In [152]:
np.uint16(4096) & 0xfff

0

In [166]:
255 << 8 | 255

65535

In [184]:
925

925

In [185]:
925 >> 8

3

In [192]:
a[0] = 925 >> 8

In [195]:
a[1] = 925

In [196]:
a[:2]

array([  3, 157], dtype=uint8)

In [197]:
a[0] << 8 | a[1]

925

In [198]:
0b11111

31

In [199]:
bin(2)

'0b10'

In [352]:
file = './src/files/loader.asm'

In [353]:
with open(file, 'r') as f:
    read_file = f.readlines()

In [354]:
read_file

['        @   0\n',
 'INIT    IO  1        ; Get Data from device\n',
 '        MM  IADDR     ; Save first byte of initial address\n',
 '        IO  1\n',
 '        MM  IADDR+1   ; Save second byte of initial address\n',
 '        IO  1\n',
 '        MM  SIZE      ; Save size\n',
 '\n',
 'LOOP    IO  1        ; Read byte from file\n',
 '        CN  2        ; Activate Indirect Mode\n',
 '        MM  IADDR     ; Indirect move to current address\n',
 '\n',
 '        LD  IADDR+1   ; Get current address\n',
 '        +   ONE       ; Sum 1\n',
 '        MM  IADDR+1   ; Put it back\n',
 '\n',
 '        LD  SIZE      ; Get size\n',
 '        -   ONE      ; Subtract 1\n',
 '        MM  SIZE      ; Put it back\n',
 '\n',
 '        JZ  END       ; Finish if size = 0\n',
 '        JP  LOOP\n',
 '\n',
 'END     OS  /F        ; End Program\n',
 '\n',
 'IADDR   K   0\n',
 '        K   0\n',
 'SIZE    K   0\n',
 'ONE     K   1\n',
 '        # INIT']

In [307]:
def prepare_line(line):
    if line == '':
        return
    return line.split(';')[0].strip()

In [355]:
codigo = [*filter(lambda line: len(line), map(lambda x: prepare_line(x).split(), read_file))]

In [356]:
codigo

[['@', '0'],
 ['INIT', 'IO', '1'],
 ['MM', 'IADDR'],
 ['IO', '1'],
 ['MM', 'IADDR+1'],
 ['IO', '1'],
 ['MM', 'SIZE'],
 ['LOOP', 'IO', '1'],
 ['CN', '2'],
 ['MM', 'IADDR'],
 ['LD', 'IADDR+1'],
 ['+', 'ONE'],
 ['MM', 'IADDR+1'],
 ['LD', 'SIZE'],
 ['-', 'ONE'],
 ['MM', 'SIZE'],
 ['JZ', 'END'],
 ['JP', 'LOOP'],
 ['END', 'OS', '/F'],
 ['IADDR', 'K', '0'],
 ['K', '0'],
 ['SIZE', 'K', '0'],
 ['ONE', 'K', '1'],
 ['#', 'INIT']]

In [310]:
linha_de_codigo = codigo[1]

In [311]:
linha_de_codigo

['INIT', 'IO', '1']

In [312]:
for linha in codigo:
    if len(linha) == 2:
        linha.insert(0, '')

In [314]:
aux = pd.DataFrame(codigo, columns=['Label', 'Operacao', 'Operador'])

In [315]:
linha_de_codigo = aux.iloc[0]

In [330]:
linha_de_codigo['Operador']

0

In [322]:
tabela.append(['', '', lin])

In [309]:
tabela = pd.DataFrame(columns=['Endereco', 'Valor', 'Codigo'])

In [331]:
linha_de_codigo

Label        
Operacao    @
Operador    0
Name: 0, dtype: object

In [343]:
if 'a':
    print(1)

1


In [409]:
import pandas as pd
import numpy as np

OPERACOES = {
    "JP": 0x0, "JZ": 0x1, "JN": 0x2, "CN": 0x3,
    "+": 0x4, "-": 0x5, "*": 0x6, "/": 0x7,
    "LD": 0x8, "MM": 0x9, "SC": 0xA, "OS": 0xB,
    "IO": 0xC,
}

TAMANHO_DAS_OPERACOES = {
    "JP": 2, "JZ": 2, "JN": 2, "CN": 1,
    "+": 2, "-": 2, "*": 2, "/": 2,
    "LD": 2, "MM": 2, "SC": 2, "OS": 1,
    "IO": 1,
}


def prepare_line(line):
    if line == '':
        return
    return line.split(';')[0].strip()


def try_to_make_int(op):
    try:
        return int(op)
    except:
        return op


class Montador(object):
    def __init__(self, file):

        with open(file, 'r') as f:
            read_file = f.readlines()

        codigo = [*filter(lambda line: len(line), map(lambda x: prepare_line(x).split(), read_file))]

        for linha in codigo:
            if len(linha) == 2:
                linha.insert(0, '')

        self.contador_de_instrucoes = 0
        self.codigo_tabelado = pd.DataFrame(codigo, columns=['Label', 'Operacao', 'Operador'])
        self.codigo_tabelado['Operador'] = self.codigo_tabelado['Operador'].apply(lambda op: try_to_make_int(op))

        self.tabela_final = pd.DataFrame(columns=['End', 'Obj', 'Linha', 'Codigo'])
        self.tabela_labels = pd.DataFrame(columns=['Label', 'Valor'])

        self.inicio_do_programa = 0

    def montar(self):
        self.primeiro_passo()

    def primeiro_passo(self):
        for index, linha_de_codigo in self.codigo_tabelado.iterrows():
            if linha_de_codigo['Label']:
                self.tabela_labels.loc[len(self.tabela_labels)] = [linha_de_codigo['Label'], hex(self.contador_de_instrucoes)]

            if linha_de_codigo['Operacao'] == '@':
                self.inicio_do_programa = self.contador_de_instrucoes = linha_de_codigo['Operador']
                self.tabela_final.loc[index] =[
                    '',
                    '',
                    index+1,
                    ' '.join(linha_de_codigo.apply(str).values).strip()
                ]
            elif linha_de_codigo['Operacao'] == '$':
                self.tabela_final.loc[index] =[
                    hex(self.contador_de_instrucoes),
                    hex(linha_de_codigo['Operador']),
                    index+1,
                    ' '.join(linha_de_codigo.apply(str).values).strip()
                ]
                self.contador_de_instrucoes += linha_de_codigo['Operador']
            elif linha_de_codigo['Operacao'] == 'K':
                self.tabela_final.loc[index] =[
                    hex(self.contador_de_instrucoes),
                    hex(linha_de_codigo['Operador']),
                    index+1,
                    ' '.join(linha_de_codigo.apply(str).values).strip()
                ]
                self.contador_de_instrucoes += 1
            elif linha_de_codigo['Operacao'] == '#':
                continue
            else:
                self.contador_de_instrucoes += TAMANHO_DAS_OPERACOES[linha_de_codigo['Operacao']]

    def segundo_passo(self):
        pass

In [410]:
montador = Montador(file='./src/files/loader.asm')

In [411]:
montador.montar()

In [412]:
montador.tabela_final

,End,Obj,Linha,Codigo
0,,,1,@ 0
19,0x1e,0x0,20,IADDR K 0
20,0x1f,0x0,21,K 0
21,0x20,0x0,22,SIZE K 0
22,0x21,0x1,23,ONE K 1


In [413]:
montador.tabela_labels

,Label,Valor
0,INIT,0x0
1,LOOP,0x9
2,END,0x1d
3,IADDR,0x1e
4,SIZE,0x20
5,ONE,0x21


## Exemplo

In [290]:
## Assembler

import logging
import glob
import os
import sys

from ctypes import c_uint8

fmt = '[{levelname:7s}] {name:s}: {message:s}'
logger = logging.getLogger(__name__)


class AssemblyError(Exception):
    pass


class Assembler:
    # (Object Code, Size)
    mnemonics_table = {
        "JP": (0x0, 2), "JZ": (0x1, 2), "JN": (0x2, 2), "CN": (0x3, 1),
        "+": (0x4, 2), "-": (0x5, 2), "*": (0x6, 2), "/": (0x7, 2),
        "LD": (0x8, 2), "MM": (0x9, 2), "SC": (0xA, 2), "OS": (0xB, 1),
        "IO": (0xC, 1),
    }

    pseudo_table = ['@', '#', '$', 'K']

    def __init__(self, filen=None, make_list=True, dump_tables=True):
        if not filen:
            raise RuntimeError('File name not provided to Assembler')

        logger.debug('Initializing Assembler')

        # Removes file extension
        self.filename = '.'.join(filen.split('.')[:-1])
        logger.debug('Base filename: %s', self.filename)

        self.labels = {}
        self.obj_code = []
        self.instruction_counter = 0
        self.initial_address = 0
        self.current_object_file = 0

        try:
            with open(filen, 'r') as f:
                all_lines = f.readlines()
        except FileNotFoundError:
            logger.error('File %s not found', filen)
            raise AssemblyError('File not found')
        except UnicodeDecodeError:
            raise AssemblyError('Certifique-se que não há letras acentuadas no arquivo!')

        logger.debug('Preprocessing file')
        self.lines = []
        for i, line in enumerate(all_lines):
            # Separate comments
            command, comment = (l.strip() for l in line.split(';')) if ';' in line else (line.strip(), '')
            if command == '' and comment == '':  # skip blank lines
                continue

            l = command.split()
            self.lines.append((i + 1, l, comment))
        logger.debug('Finished preprocessing')

        self.list_file = None
        if (make_list):
            self.list_file = self.filename + '.lst'

            with open(self.list_file, 'w') as f:
                print('{} LIST FILE'.format(self.list_file), file=f)
                print('{}-----------'.format('-' * len(self.list_file)), file=f)
                print('ADDRESS   OBJECT    LINE   SOURCE', file=f)

        self.lb_table_file = None
        if (dump_tables):
            self.lb_table_file = self.filename + '.asm.labels'

            with open(self.lb_table_file, 'w') as f:
                print('{} LABEL TABLE FILE'.format(self.lb_table_file), file=f)
                print('{}-----------------'.format('-' * len(self.lb_table_file)), file=f)
                print('LABEL           VALUE', file=f)

        self.obj_file = self.filename + '.obj'
        for f in glob.glob(self.obj_file + '.*'):
            os.remove(f)

    def assemble(self):
        for self.step in [1, 2]:
            logger.debug('Initializing Step %d', self.step)

            self.instruction_counter = 0
            for lineno, code, comment in self.lines:
                # logger.debug('Processing line %d', lineno)

                if len(code) == 0:  # Comment only, do nothing on first step
                    if self.step == 2:
                        self.list(line=lineno, comment=comment)

                elif len(code) == 1:  # Label only
                    if self.step == 2:
                        self.list(line=lineno, comment=comment, code=code, address=self.instruction_counter)
                        continue

                    label = code[0]
                    if label in [*self.mnemonics_table, *self.pseudo_table]:  # Lonely operation
                        raise AssemblyError(
                            'Assembly error on line {:d} "{}": operation must have operator'.format(lineno,
                                                                                                    ' '.join(code)))

                    if label in self.labels and self.labels[label] is not None:  # Label already defined
                        raise AssemblyError(
                            'Assembly error on line {:d} label "{}" already defined'.format(lineno, label))

                    self.labels[label] = self.instruction_counter

                elif len(code) == 2:  # Operation and Operator
                    self.process_code(lineno, code, comment)

                elif len(code) == 3:  # Label, Operation and Operator
                    if self.step == 1:
                        label = code[0]
                        if label in [*self.mnemonics_table, *self.pseudo_table]:  # First element should be label
                            raise AssemblyError(
                                'Assembly error on line {:d} "{}": operation on label position'.format(lineno,
                                                                                                       ' '.join(code)))

                        if label in self.labels and self.labels[label] is not None:  # Label already defined
                            raise AssemblyError(
                                'Assembly error on line {:d} label "{}" already defined'.format(lineno, label))

                        self.labels[label] = self.instruction_counter

                    self.process_code(lineno, code, comment)

                else:
                    raise AssemblyError('Assembly error on line {:d}: Too many things!!'.format(lineno))

            logger.debug('Finished Step %d', self.step)
            self.dump_label_table()
            self.save_obj()

    def process_code(self, lineno, code, comment):
        print('Aquiiiiii')
        print(code)
        operation = code[-2]
        if operation not in [*self.mnemonics_table, *self.pseudo_table]:  # Unknown operation
            raise AssemblyError('Assembly error on line {:d}: Unknown operation "{}"'.format(lineno, operation))

        try:
            operator = int(code[-1])  # try decimal
        except ValueError:
            try:
                if code[-1][0] == '/':
                    operator = int(code[-1][1:], 16)  # try hex
                else:
                    raise
            except ValueError:  # label
                lb = code[-1].partition('+') if '+' in code[-1] else code[-1].partition('-') if '-' in code[-1] else [
                    code[-1]]
                if self.step == 1:
                    if lb[0] not in self.labels:
                        self.labels[lb[0]] = None
                    operator = None
                elif self.step == 2:
                    if self.labels[lb[0]] is None:
                        raise AssemblyError(
                            'Assembly error on line {:d}: undefined label "{}"'.format(lineno, code[-1]))
                    if len(lb) == 1:
                        operator = self.labels[lb[0]]
                    else:
                        operator = self.labels[lb[0]] + int(lb[2]) if lb[1] == '+' else self.labels[lb[0]] - int(lb[2])
                        
        print('Operator: ', operator)
        print('Operation: ', operation)

        # Pseudo instruction
        if operation in self.pseudo_table:
            if operation != '#' and type(operator) != type(int()):
                raise AssemblyError(
                    'Assembly error on line {:d}: {} operator must be an integer!'.format(lineno, operation))

            if operation == '@':
                if operator > 0xFFFF:
                    raise AssemblyError(
                        'Assembly error on line {:d}: operator out of range "{:0x}"'.format(lineno, operator))
                # elif operator < 0x0100:
                #     raise AssemblyError('Assembly error on line {:d}: memory area before 0x0100 should never be accessed'.format(lineno))
                self.list(line=lineno, code=code, comment=comment)
                if self.step == 2 and self.instruction_counter != None:  # New code block, save current
                    self.save_obj()
                    self.obj_code = []
                self.initial_address = operator
                self.instruction_counter = operator

            elif operation == '$':
                if operator > 0xFFF:
                    raise AssemblyError(
                        'Assembly error on line {:d}: operator out of range "{:0x}"'.format(lineno, operator))
                self.list(line=lineno, code=code, comment=comment, address=self.instruction_counter)
                self.instruction_counter += operator
                if self.step == 2:
                    self.obj_code.extend([c_uint8(0)] * operator)

            elif operation == 'K':
                if operator > 0xFF:
                    raise AssemblyError(
                        'Assembly error on line {:d}: operator out of range "{:0x}"'.format(lineno, operator))
                self.list(line=lineno, code=code, comment=comment, address=self.instruction_counter, object=operator)
                if self.step == 2:
                    self.obj_code.append(c_uint8(operator))
                self.instruction_counter += 1

            elif self.step == 2 and operation == '#':
                if operator > 0xFFFF:
                    raise AssemblyError(
                        'Assembly error on line {:d}: operator out of range "{:0x}"'.format(lineno, operator))
                self.list(line=lineno, code=code, comment=comment)
                self.save_obj()
                self.initial_address = 0x0022
                self.obj_code = [c_uint8(operator >> 8), c_uint8(operator & 0xFF)]

        # Normal instruction
        else:
            instruction_size = self.mnemonics_table[operation][1]
            if self.step == 1:
                self.instruction_counter += instruction_size
                return

            if instruction_size == 1:
                obj_code = self.mnemonics_table[operation][0] << 4 | operator
                self.obj_code.append(c_uint8(obj_code))
            elif instruction_size == 2:
                obj_code = self.mnemonics_table[operation][0] << 12 | operator
                self.obj_code.append(c_uint8(obj_code >> 8))
                self.obj_code.append(c_uint8(obj_code & 0xFF))

            self.list(line=lineno, code=code, comment=comment, address=self.instruction_counter, object=obj_code)
            self.instruction_counter += instruction_size

    def list(self, **kwargs):
        if self.list_file is None or self.step != 2:
            return

        comm = '; {:s}'.format(kwargs['comment']) if 'comment' in kwargs and kwargs['comment'] != '' else ''
        addr = '{:-04X}'.format(kwargs['address']) if 'address' in kwargs else '    '
        obj = '{:-6X}'.format(kwargs['object']) if 'object' in kwargs else '      '
        code = '{:s} '.format(' '.join(kwargs['code'])) if 'code' in kwargs else ''
        line = '{:-4d}'.format(kwargs['line']) if 'line' in kwargs else '    '

        with open(self.list_file, 'a') as f:
            print('   {}   {}    {}   {}{}'.format(
                addr, obj, line, code, comm), file=f)

    def dump_label_table(self):
        if self.lb_table_file is None or self.step != 1:
            return

        with open(self.lb_table_file, 'a') as f:
            for label in self.labels:
                print('{:<15s}  {:>04X}'.format(label, self.labels[label]), file=f)

    def save_obj(self):
        if self.step != 2:
            return

        logger.debug('Saving object')

        obj_codes = [self.obj_code[x:x + 0xFF] for x in range(0, len(self.obj_code), 0xFF)]  # Split into FF

        pc = self.initial_address
        for obj in obj_codes:
            chk = c_uint8(0xFF)
            with open(self.obj_file + '.{:d}'.format(self.current_object_file), 'w') as f:
                with open(self.obj_file + '.bin.{:d}'.format(self.current_object_file), 'wb') as fb:
                    fb.write(pc.to_bytes(2, 'big'))
                    fb.write((len(obj) & 0xFF).to_bytes(1, 'big'))
                    print('{:02X} {:02X}'.format(pc >> 8, pc & 0xFF), end=' ', file=f)
                    print('{:02X}'.format(len(obj)), end=' ', file=f)

                    i = 3
                    for o in obj:
                        fb.write((o.value & 0xFF).to_bytes(1, 'big'))
                        print('{:02X}'.format(o.value), end=' ', file=f)

                        chk.value ^= o.value
                        i += 1
                        pc += 1
                        if i % 16 == 0:  # Break line every few elements to improve readability
                            print('', file=f)

                    fb.write((chk.value & 0xFF).to_bytes(1, 'big'))
                    print('{:02X}'.format(chk.value), end='', file=f)

                    self.current_object_file += 1


In [299]:
assemblier = Assembler(filen='./src/files/loader.asm')

In [300]:
assemblier.assemble()

Aquiiiiii
['@', '/0000']
Operator:  0
Operation:  @
Aquiiiiii
['INIT', 'IO', '/1']
Operator:  1
Operation:  IO
Aquiiiiii
['MM', 'IADDR']
Operator:  None
Operation:  MM
Aquiiiiii
['IO', '/1']
Operator:  1
Operation:  IO
Aquiiiiii
['MM', 'IADDR+1']
Operator:  None
Operation:  MM
Aquiiiiii
['IO', '/1']
Operator:  1
Operation:  IO
Aquiiiiii
['MM', 'SIZE']
Operator:  None
Operation:  MM
Aquiiiiii
['LOOP', 'IO', '/1']
Operator:  1
Operation:  IO
Aquiiiiii
['CN', '/2']
Operator:  2
Operation:  CN
Aquiiiiii
['MM', 'IADDR']
Operator:  None
Operation:  MM
Aquiiiiii
['LD', 'IADDR+1']
Operator:  None
Operation:  LD
Aquiiiiii
['+', 'ONE']
Operator:  None
Operation:  +
Aquiiiiii
['MM', 'IADDR+1']
Operator:  None
Operation:  MM
Aquiiiiii
['LD', 'SIZE']
Operator:  None
Operation:  LD
Aquiiiiii
['-', 'ONE']
Operator:  None
Operation:  -
Aquiiiiii
['MM', 'SIZE']
Operator:  None
Operation:  MM
Aquiiiiii
['JZ', 'END']
Operator:  None
Operation:  JZ
Aquiiiiii
['JP', 'LOOP']
Operator:  None
Operation:  JP
A

In [301]:
assemblier.labels

{'END': 29, 'IADDR': 30, 'INIT': 0, 'LOOP': 9, 'ONE': 33, 'SIZE': 32}

In [296]:
assemblier.dump_label_table()

In [298]:
0x1E

30